<a href="https://colab.research.google.com/github/wlgns222/ROKA/blob/main/ai-study/deep-learning-from-scratch-vol1/Ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ch5. 오차역전파법**

오차역전파법 (Backpropagation) : 가중치 매개변수의 기울기를 효율적으로 계산하는 방법

## 5.1 계산 그래프

- 순전파 : 계산을 오른쪽에서 왼쪽으로 진행
- 역전파 : 오른쪽에서 왼쪽으로 미분을 전달

국소적 계산 : 전체에서 어떤 일이 벌어지든 상관 없이 자신과 관계된 정보만을 출력

## 5.2 연쇄법칙

## 5.3 역전파

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [1]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out

  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

### 5.4.2 덧셈 계층

In [ ]:
class AddLayer:
  def __init__(self) :
    pass
  def forward(self, x, y):
    out = x + y
    return out
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1

### 5.4 시스템 조립

In [ ]:
# 1. 환경 설정 (데이터)
apple = 100; apple_num = 2
orange = 150; orange_num = 3
tax = 1.1

# 2. 계층(Layer) 생성
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 3. 순전파 (Forward)
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# 4. 역전파 (Backward)
dprice = 1 # 시작 미분값
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print(f"최종 가격: {int(price)}")
print(f"사과 개수에 대한 미분: {dapple_num}")
print(f"귤 가격에 대한 미분: {dorange}")
print(f"소비세에 대한 미분: {dtax}")

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

**ReLU의 순전파 (Forward)**

ReLU는 입력 x 가 0보다 크면 그대로 내보내고, 0 이하면 0을 내보낸다.

$$y = \begin{cases} x & (x > 0) \\ 0 & (x \le 0) \end{cases}$$


**ReLU의 역전파 (Backward)**
- 순전파 때 x > 0 이었다면 : 미분값이 1 이다. 즉 뒤에서 온 미분값을 그대로 전달한다.
- 순전파 때 x < 0 이었다면 : 미분값이 0 이다. 즉, 뒤에서 온 미분값이 무엇이든 0 을 전달한다.

$$\frac{\partial y}{\partial x} = \begin{cases} 1 & (x > 0) \\ 0 & (x \le 0) \end{cases}$$

In [4]:
import numpy as np

class ReLU:
  def __init__(self) :
    self.mask = None

  def forward(self, x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0
    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx


인스턴스 변수 **mask**
- True/False 로 구성된 넘파이 배열
- 어떤 원소가 0 이하였는지 기억하는 변수

In [5]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [6]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


**ReLU 기울기 차단 이유**
- 기울기 소실 방지 : 0보다 큰 영역에서 기울기가 항상 1이기 때문에, 층이 깊어져도 신호가 약해지지 않고 끝까지 전달된다.
- 신경망의 전원 스위치 : x<=0 일때 기울기를 0으로 만드는 것은 해당 뉴런이 현재 학습에 기여하지 않도록 전원을 끄는것과 같다. 이는 모델이 불필요한 정보에 반응하지 않도록 하는 효과를 준다.

### 5.5.2 Sigmoid 계층